In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class AdvancedCNN_MP(nn.Module):
    def __init__(self):
        super(AdvancedCNN_MP, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, padding=1)  
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, padding=1)  
        self.bn2 = nn.BatchNorm2d(20)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(500, 64)  # 입력 크기를 조정
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [2]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, ConcatDataset

# 데이터 로딩 및 전처리
def load_mnist_data(train_dirs, test_dir):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    train_datasets = [datasets.ImageFolder(root=dir, transform=transform) for dir in train_dirs]
    train_dataset = ConcatDataset(train_datasets)
    test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    return train_loader, test_loader



In [3]:
import torch
import torch.optim as optim
import torch.nn.functional as F

def train_and_evaluate(model, train_loader, test_loader, epochs=10, device='cuda'):
    optimizer = optim.Adam(model.parameters())
    criterion = torch.nn.CrossEntropyLoss()

    model.to(device)

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            if batch_idx % 100 == 0:
                print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.4f}%)]\tLoss: {loss.item():.6f}')

        train_loss /= len(train_loader.dataset)
        train_accuracy = 100. * correct / total
        print(f'\nTraining set: Average loss: {train_loss:.4f}, Accuracy: {correct}/{total} ({train_accuracy:.0f}%)\n')

        # Evaluate on test data
        model.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += criterion(output, target).item()
                _, predicted = torch.max(output.data, 1)
                correct += (predicted == target).sum().item()

        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({test_accuracy:.4f}%)\n')

In [4]:
# 실행
train_dirs = ['C:/Users/koo/my_workspace/privacy_term_project/random_pertubation_image_by_adv_mp', 
             'C:/Users/koo/my_workspace/privacy_term_project/mnist_btm_hash/training' ]

test_dir = 'C:/Users/koo/my_workspace/privacy_term_project/mnist_btm_hash/testing'
model = AdvancedCNN_MP()
train_loader, test_loader = load_mnist_data(train_dirs, test_dir)
train_and_evaluate(model, train_loader, test_loader, epochs=10)
torch.save(model.state_dict(), 'advanced_mnist_cnn_by_ADT_mp.pth') # 모델 저장

Train Epoch: 0 [0/68657 (0.0000%)]	Loss: 2.354144
Train Epoch: 0 [6400/68657 (9.3197%)]	Loss: 0.312330
Train Epoch: 0 [12800/68657 (18.6393%)]	Loss: 0.225745
Train Epoch: 0 [19200/68657 (27.9590%)]	Loss: 0.122686
Train Epoch: 0 [25600/68657 (37.2787%)]	Loss: 0.145004
Train Epoch: 0 [32000/68657 (46.5983%)]	Loss: 0.104238
Train Epoch: 0 [38400/68657 (55.9180%)]	Loss: 0.127613
Train Epoch: 0 [44800/68657 (65.2377%)]	Loss: 0.063038
Train Epoch: 0 [51200/68657 (74.5573%)]	Loss: 0.064381
Train Epoch: 0 [57600/68657 (83.8770%)]	Loss: 0.098379
Train Epoch: 0 [64000/68657 (93.1966%)]	Loss: 0.064855

Training set: Average loss: 0.0028, Accuracy: 65021/68657 (95%)

Test set: Average loss: 0.0008, Accuracy: 9843/10000 (98.4300%)

Train Epoch: 1 [0/68657 (0.0000%)]	Loss: 0.080402
Train Epoch: 1 [6400/68657 (9.3197%)]	Loss: 0.044954
Train Epoch: 1 [12800/68657 (18.6393%)]	Loss: 0.137365
Train Epoch: 1 [19200/68657 (27.9590%)]	Loss: 0.068722
Train Epoch: 1 [25600/68657 (37.2787%)]	Loss: 0.060952
Tra